In [1]:
import pandas as pd
#handling data
import pandas as pd
import numpy as np
from scipy import stats
from operator import itemgetter

#import twitter credentials
consumer_key = "jjSz1RE4ftTNmqB1XuUTM22Fc"
consumer_secret = "5SNWrhQStzMp3UDwVY7YGuEofQ4QOBBP4rOo4hGnhKpdFQoVi9"
access_token = "68979886-IzdibLmYAx39y8PLNWA7kLPKl2rTDlLPCnf557I45"
access_token_secret = "tjVsF4mx4vS9JO0hPcS7b8qoP7oIZK1A8nX0aMwhkNEDG"

#handling information
import re
import json

#handling plots
import matplotlib.pyplot as plt
import seaborn as sns

#for network creation
import networkx as nx
import tweepy
import math
from collections import Counter

In [2]:
from pymongo import MongoClient

myclient = MongoClient()
nexus = myclient["nexus"]
tweets = nexus["tweets"]
users = nexus["users"]
reports = nexus["reports"]

In [35]:
reps = reports.find({'user.sub': 'google-oauth2|110224375451341506987'})

hashtags_list = []
for r in reps:
    hashtags_list.append(r['hashtags'])

In [110]:
unique_data = [list(x) for x in set(tuple(sorted(x)) for x in hashtags_list)]

In [123]:
for hashtags in unique_data: 
    queries = []
    for hashtag in hashtags:
        hashtag = hashtag[1:]
        queries.append({"entities.hashtags.text": hashtag})
    query = [{'$match': {'$or': queries}}]
    tweets = nexus.tweets.aggregate(query)
    
    tweet_list = []
    for tweet in tweets:
        tweet_list.append(tweet)
        
    tweets_df = pd.DataFrame(tweet_list)
    
    all_hashtags = []
    for e in tweets_df.entities:
        hashtags = [t['text'] for t in e['hashtags']]
        all_hashtags = all_hashtags + hashtags

    from collections import Counter
    c = Counter(all_hashtags)
    most_common_tuples = c.most_common()
    sorted_keys = sorted(c, key=c.get, reverse=True)
    
    hashtag_dict = {}
    for t in most_common_tuples:
        hashtag_dict[t[0]] = t[1]
        
    hashtag_wordclouds = []
    for t in most_common_tuples:
        hashtag_wordclouds.append({'text': t[0], 'value': t[1]})

    for t in most_common_tuples:
        tweets_df['#'+t[0]] = False

    for i, e in enumerate(tweets_df.entities):
        hashtags = [t['text'] for t in e['hashtags']]
        for hashtag in hashtags:
            tweets_df.at[i, '#' + hashtag] = True

    
    def apply_func(x):
        if not isinstance(x, float):
            if 'full_text' in x: 
                return x['full_text']
            else:
                return float('nan')
        else:
            return float('nan')


    tweets_df['full_text'] = tweets_df['extended_tweet'].apply(lambda x: apply_func(x))
    tweets_df.full_text.fillna(tweets_df.text, inplace=True)


    import re 
    from collections import Counter
    import requests

    import extraction
    import requests

    import favicon



    url_list = []
    for text in tweets_df['text'].values.tolist():
        urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
        url_list += urls



    url_list = list(filter(lambda url: len(url)>13, url_list))

    c = Counter(url_list)
    most_common_urls = c.most_common()
    sorted_urls = sorted(c, key=c.get, reverse=True)

    news_articles = []
    twitter_domain_url = 'https://twitter.com'

    url_list = []
    for e in tweets_df.entities:
        for url_obj in e['urls']:
            if url_obj['expanded_url'][:19] != 'https://twitter.com':
                url_list.append(url_obj['expanded_url'])

    c = Counter(url_list)
    most_common_urls = c.most_common()
    sorted_urls = sorted(c, key=c.get, reverse=True)

    for i, url in enumerate(sorted_urls):
        print('count value:', i)
        if i == 20:
            break
        try:
            news_article_dict = {}

            html = requests.get(url).text
            extracted = extraction.Extractor().extract(html, source_url=url)
            icon_url = favicon.get(url)[0][0]



            if url[:19] != twitter_domain_url:
                print('title:', extracted.title)
                print('description:', extracted.description)
                print(extracted.image, url, icon_url)



                news_article_dict['title'] = extracted.title
                news_article_dict['description'] = extracted.description
                news_article_dict['favicon'] = icon_url
                news_article_dict['image'] = extracted.image
                news_article_dict['url'] = url
                news_article_dict['share_count'] = most_common_urls[i][1]

                news_articles.append(news_article_dict)





            print('\n')
        except:
            print(url)
            print('\n')




    #Accesing twitter from the App created in my account
    def autorize_twitter_api():
        """
        This function gets the consumer key, consumer secret key, access token 
        and access token secret given by the app created in your Twitter account
        and authenticate them with Tweepy.
        """
        #Get access and costumer key and tokens
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)

        return auth

    api = tweepy.API(wait_on_rate_limit_notify=True,wait_on_rate_limit=True, auth_handler=autorize_twitter_api())


    retweeted_ids = []
    for rt in tweets_df['retweeted_status'].values:
        if not isinstance(rt, float):
            retweeted_ids.append(rt['id'])




    c = Counter(retweeted_ids)
    most_common_tuples = c.most_common()
    sorted_keys = sorted(c, key=c.get, reverse=True)
    most_common_tuples

    ids = []
    for t in most_common_tuples:
        ids.append(t[0])
    ids = ids[:20]

    statuses = api.statuses_lookup(ids)

    statuses_list = []
    for status in statuses:
        t_obj = {
            'text':status._json['text'], 
            'id':status._json['id'], 
            'tweet_link': 'https://twitter.com/i/web/status/' + status._json['id_str'], 
            'user_screen_name':status._json['user']['screen_name'],
            'json': status._json
        }

        t = (t_obj , status._json['retweet_count'])
        statuses_list.append(t)


    s = sorted(statuses_list, key=lambda x: x[1])
    s.reverse() 

    viral_tweets = [x[0] for x in s]
    
    # Create a second dataframe to put important information
    tweets_final = pd.DataFrame(columns = ["created_at", "id", "in_reply_to_screen_name", "in_reply_to_status_id", "in_reply_to_user_id",
                                          "retweeted_id", "retweeted_screen_name", "user_mentions_screen_name", "user_mentions_id", 
                                           "text", "user_id", "screen_name", "followers_count"])

    # Columns that are going to be the same
    equal_columns = ["created_at", "id", "text"]
    tweets_final[equal_columns] = tweets_df[equal_columns]

    # Get the basic information about user 
    def get_basics(tweets_final):
        print(tweets_df["user"])
        tweets_final["screen_name"] = tweets_df["user"].apply(lambda x: x["screen_name"])
        tweets_final["user_id"] = tweets_df["user"].apply(lambda x: x["id"])
        tweets_final["followers_count"] = tweets_df["user"].apply(lambda x: x["followers_count"])
        return tweets_final

    # Get the user mentions 
    def get_usermentions(tweets_final):
        # Inside the tag 'entities' will find 'user mentions' and will get 'screen name' and 'id'
        tweets_final["user_mentions_screen_name"] = tweets_df["entities"].apply(lambda x: x["user_mentions"][0]["screen_name"] if x["user_mentions"] else np.nan)
        tweets_final["user_mentions_id"] = tweets_df["entities"].apply(lambda x: x["user_mentions"][0]["id_str"] if x["user_mentions"] else np.nan)
        return tweets_final

    # Get retweets
    def get_retweets(tweets_final):
        # Inside the tag 'retweeted_status' will find 'user' and will get 'screen name' and 'id'    
        tweets_final["retweeted_screen_name"] = tweets_df["retweeted_status"].apply(lambda x: x["user"]["screen_name"] if x is not np.nan else np.nan)
        tweets_final["retweeted_id"] = tweets_df["retweeted_status"].apply(lambda x: x["user"]["id_str"] if x is not np.nan else np.nan)
        return tweets_final

    # Get the information about replies
    def get_in_reply(tweets_final):
        # Just copy the 'in_reply' columns to the new dataframe
        tweets_final["in_reply_to_screen_name"] = tweets_df["in_reply_to_screen_name"]
        tweets_final["in_reply_to_status_id"] = tweets_df["in_reply_to_status_id"]
        tweets_final["in_reply_to_user_id"]= tweets_df["in_reply_to_user_id"]
        return tweets_final

    # Lastly fill the new dataframe with the important information
    def fill_df(tweets_final):
        get_basics(tweets_final)
        get_usermentions(tweets_final)
        get_retweets(tweets_final)
        get_in_reply(tweets_final)
        return tweets_final

    # Get the interactions between the different users
    def get_interactions(row):
        # From every row of the original dataframe
        # First we obtain the 'user_id' and 'screen_name'
        user = row["user_id"], row["screen_name"]
        # Be careful if there is no user id
        if user[0] is None:
            return (None, None), []

        # The interactions are going to be a set of tuples
        interactions = set()

        # Add all interactions 
        # First, we add the interactions corresponding to replies adding the id and screen_name
        interactions.add((row["in_reply_to_user_id"], row["in_reply_to_screen_name"]))
        # After that, we add the interactions with retweets
        interactions.add((row["retweeted_id"], row["retweeted_screen_name"]))
        # And later, the interactions with user mentions
        interactions.add((row["user_mentions_id"], row["user_mentions_screen_name"]))

        # Discard if user id is in interactions
        interactions.discard((row["user_id"], row["screen_name"]))
        # Discard all not existing values
        interactions.discard((None, None))
        # Return user and interactions
        return user, interactions

    tweets_final = fill_df(tweets_final)


    tweets_final = tweets_final.where((pd.notnull(tweets_final)), None)

    graph = nx.Graph()

    for index, tweet in tweets_final.iterrows():
        user, interactions = get_interactions(tweet)
        user_id, user_name = user
        tweet_id = int(tweet["id"])
        #tweet_sent = tweet["sentiment"]
        for interaction in interactions:
            int_id, int_name = interaction
            graph.add_edge(user_id, int_id, tweet_id=tweet_id)


            graph.node[user_id]["name"] = user_name
            graph.node[user_id]["text"] = tweet['text']
            graph.node[int_id]["name"] = int_name
            graph.node[int_id]["text"] = tweet['text']

    degrees = [val for (node, val) in graph.degree()]

    largest_subgraph = max(nx.connected_component_subgraphs(graph), key=len)

    graph_centrality = nx.degree_centrality(largest_subgraph)

    max_de = max(graph_centrality.items(), key=itemgetter(1))

    graph_closeness = nx.closeness_centrality(largest_subgraph)

    max_clo = max(graph_closeness.items(), key=itemgetter(1))

    graph_betweenness = nx.betweenness_centrality(largest_subgraph, normalized=True, endpoints=False)

    max_bet = max(graph_betweenness.items(), key=itemgetter(1))

    all_bet = sorted(graph_betweenness.items(), key=itemgetter(1))

    all_bet.reverse()

    all_de = sorted(graph_centrality.items(), key=itemgetter(1))
    all_de.reverse()

    ids = []
    for de in all_de:
    #     print(graph.node[de[0]])
        ids.append(de[0])
    #     print(de[0])
    #     print(graph.node[de[0]]['name'])

    ids = ids[:10]

    users = api.lookup_users(user_ids=ids)


    user_list = []
    for user in users:
        user_list.append(user._json)

    ##############################################
    influencers = user_list
    ##############################################

    print(hashtag_wordclouds)
    print(news_articles)
    print(viral_tweets)
    print(influencers)

count value: 0
title: Insurtech
description: None
None https://insurtechnttdata.everis.com/?lang=en&utm_campaign=Insurtech&utm_medium=email&utm_source=Revue%20newsletter https://insurtechnttdata.everis.com/favicon.ico


count value: 1
title: Riskcovry closes pre-series A round on INR 9.3 crore ($1.2m) | FinTech Global
description: Digital Insurance platform Riskcovry has reportedly closed its pre-Series A round on INR 9.3 crore ($1.2m).
https://member.fintech.global/wp-content/uploads/sites/3/2020/05/photo-1525466760727-1d8be8721154.jpg https://bit.ly/3d4YZBd https://analysis.cairns.cl/wp-content/uploads/2017/04/FG-Logo_Black-BG_Horizontal.png


count value: 2
title: Fostering Team Morale During the Pandemic | Ansafone Contact Centers
description: Positive morale is an important element during a national crisis. While most people are distressed, it is the leaders’ role to keep the business foundation stable, its staff morally sound, and operations smoothly. There is no doubt that the C

In [ ]:
all_hashtags = []
for e in tweets_df.entities:
    hashtags = [t['text'] for t in e['hashtags']]
    all_hashtags = all_hashtags + hashtags

from collections import Counter
c = Counter(all_hashtags)
most_common_tuples = c.most_common()
sorted_keys = sorted(c, key=c.get, reverse=True)
# most_common_tuples

In [50]:
hashtag_dict = {}
for t in most_common_tuples:
    hashtag_dict[t[0]] = t[1]

In [51]:
hashtag_wordclouds = []
for t in most_common_tuples:
    hashtag_wordclouds.append({'text': t[0], 'value': t[1]})

In [90]:
##########################
hashtag_wordclouds
#########################

In [52]:
for t in most_common_tuples:
    tweets_df['#'+t[0]] = False

In [53]:
for i, e in enumerate(tweets_df.entities):
    hashtags = [t['text'] for t in e['hashtags']]
    for hashtag in hashtags:
        tweets_df.at[i, '#' + hashtag] = True

In [91]:
for t in tweets_df[tweets_df['#insurance'] == True]['text'].values:
    print(t)

In [55]:
import math
def apply_func(x):
    if not isinstance(x, float):
        if 'full_text' in x: 
            return x['full_text']
        else:
            return float('nan')
    else:
        return float('nan')
    

tweets_df['full_text'] = tweets_df['extended_tweet'].apply(lambda x: apply_func(x))
tweets_df.full_text.fillna(tweets_df.text, inplace=True)

In [121]:
url_list = []
for e in tweets_df.entities:
    for url_obj in e['urls']:
        if url_obj['expanded_url'][:19] != 'https://twitter.com':
            url_list.append(url_obj['expanded_url'])
            
c = Counter(url_list)
most_common_urls = c.most_common()
sorted_urls = sorted(c, key=c.get, reverse=True)

for i, url in enumerate(sorted_urls):
    print('count value:', i)
    if i == 20:
        break
    try:
        news_article_dict = {}
        
        html = requests.get(url).text
        extracted = extraction.Extractor().extract(html, source_url=url)
        icon_url = favicon.get(url)[0][0]
        
        
        
        if url[:19] != twitter_domain_url:
            print('title:', extracted.title)
            print('description:', extracted.description)
            print(extracted.image, url, icon_url)



            news_article_dict['title'] = extracted.title
            news_article_dict['description'] = extracted.description
            news_article_dict['favicon'] = icon_url
            news_article_dict['image'] = extracted.image
            news_article_dict['url'] = url
            news_article_dict['share_count'] = most_common_urls[i][1]

            news_articles.append(news_article_dict)
        
        
        
        
        
        print('\n')
    except:
        print(url)
        print('\n')

count value: 0
title: Insurtech
description: None
None https://insurtechnttdata.everis.com/?lang=en&utm_campaign=Insurtech&utm_medium=email&utm_source=Revue%20newsletter https://insurtechnttdata.everis.com/favicon.ico


count value: 1
title: Riskcovry closes pre-series A round on INR 9.3 crore ($1.2m) | FinTech Global
description: Digital Insurance platform Riskcovry has reportedly closed its pre-Series A round on INR 9.3 crore ($1.2m).
https://member.fintech.global/wp-content/uploads/sites/3/2020/05/photo-1525466760727-1d8be8721154.jpg https://bit.ly/3d4YZBd https://analysis.cairns.cl/wp-content/uploads/2017/04/FG-Logo_Black-BG_Horizontal.png


count value: 2
title: 403 Forbidden
description: You don't have permission to access /fostering-team-morale-during-the-pandemic/ on this server.
None https://hubs.ly/H0qwNSJ0 https://ansafone.com/wp-content/uploads/2017/11/cropped-favicon-01-270x270.png


count value: 3
title: 403 Forbidden
description: None
None http://ow.ly/DKTL30gJavw https:

In [56]:
import re 
from collections import Counter
import requests

import extraction
import requests

import favicon



url_list = []
for text in tweets_df['text'].values.tolist():
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    url_list += urls

    

url_list = list(filter(lambda url: len(url)>13, url_list))

c = Counter(url_list)
most_common_urls = c.most_common()
sorted_urls = sorted(c, key=c.get, reverse=True)

news_articles = []
twitter_domain_url = 'https://twitter.com'

for i, url in enumerate(sorted_urls):
    print('count value:', i)
    if i == 20:
        break
    resp = requests.head(url)
    try:
        news_article_dict = {}
        
        url = resp.headers["Location"]
        html = requests.get(url).text
        extracted = extraction.Extractor().extract(html, source_url=url)
        icon_url = favicon.get(url)[0][0]
        
        
        
        if url[:19] != twitter_domain_url:
            print('title:', extracted.title)
            print('description:', extracted.description)
            print(extracted.image, url, icon_url)



            news_article_dict['title'] = extracted.title
            news_article_dict['description'] = extracted.description
            news_article_dict['favicon'] = icon_url
            news_article_dict['image'] = extracted.image
            news_article_dict['url'] = url
            news_article_dict['share_count'] = most_common_urls[i][1]

            news_articles.append(news_article_dict)
        
        
        
        
        
        print('\n')
    except:
        print(url)
        print('\n')
        
        




count value: 0
title: Insurtech
description: None
None https://insurtechnttdata.everis.com/?lang=en&utm_campaign=Insurtech&utm_medium=email&utm_source=Revue%20newsletter https://insurtechnttdata.everis.com/favicon.ico


count value: 1


count value: 2


count value: 3
title: Riskcovry closes pre-series A round on INR 9.3 crore ($1.2m) | FinTech Global
description: Digital Insurance platform Riskcovry has reportedly closed its pre-Series A round on INR 9.3 crore ($1.2m).
https://member.fintech.global/wp-content/uploads/sites/3/2020/05/photo-1525466760727-1d8be8721154.jpg https://bit.ly/3d4YZBd https://analysis.cairns.cl/wp-content/uploads/2017/04/FG-Logo_Black-BG_Horizontal.png


count value: 4


count value: 5
title: 403 Forbidden
description: You don't have permission to access /fostering-team-morale-during-the-pandemic/ on this server.
None https://hubs.ly/H0qwNSJ0 https://ansafone.com/wp-content/uploads/2017/11/cropped-favicon-01-270x270.png


count value: 6


count value: 7


count

In [93]:
#######################
news_articles
#######################

In [57]:
#Import modules
%matplotlib inline

#handling data
import pandas as pd
import numpy as np
from scipy import stats
from operator import itemgetter

#import twitter credentials
consumer_key = "jjSz1RE4ftTNmqB1XuUTM22Fc"
consumer_secret = "5SNWrhQStzMp3UDwVY7YGuEofQ4QOBBP4rOo4hGnhKpdFQoVi9"
access_token = "68979886-IzdibLmYAx39y8PLNWA7kLPKl2rTDlLPCnf557I45"
access_token_secret = "tjVsF4mx4vS9JO0hPcS7b8qoP7oIZK1A8nX0aMwhkNEDG"

#handling information
import re
import json

#handling plots
import matplotlib.pyplot as plt
import seaborn as sns

#for network creation
import networkx as nx
import tweepy

In [58]:
#Accesing twitter from the App created in my account
def autorize_twitter_api():
    """
    This function gets the consumer key, consumer secret key, access token 
    and access token secret given by the app created in your Twitter account
    and authenticate them with Tweepy.
    """
    #Get access and costumer key and tokens
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    return auth

api = tweepy.API(wait_on_rate_limit_notify=True,wait_on_rate_limit=True, auth_handler=autorize_twitter_api())

In [59]:
from collections import Counter

retweeted_ids = []
for rt in tweets_df['retweeted_status'].values:
    if not isinstance(rt, float):
        retweeted_ids.append(rt['id'])




c = Counter(retweeted_ids)
most_common_tuples = c.most_common()
sorted_keys = sorted(c, key=c.get, reverse=True)
most_common_tuples

[(1265525150059048965, 2),
 (1265630172021239808, 1),
 (1265635228837650432, 1),
 (1265326626201636865, 1),
 (1260535367851270144, 1),
 (1265653585389457409, 1),
 (1265669226955587587, 1),
 (1265665331617583104, 1)]

In [60]:
ids = []
for t in most_common_tuples:
    ids.append(t[0])
ids = ids[:20]

In [62]:
statuses = api.statuses_lookup(ids)

In [63]:
statuses_list = []
for status in statuses:
    t_obj = {
        'text':status._json['text'], 
        'id':status._json['id'], 
        'tweet_link': 'https://twitter.com/i/web/status/' + status._json['id_str'], 
        'user_screen_name':status._json['user']['screen_name'],
        'json': status._json
    }
    
    t = (t_obj , status._json['retweet_count'])
    statuses_list.append(t)
    
    
s = sorted(statuses_list, key=lambda x: x[1])
s.reverse()   

In [64]:
#########################################
viral_tweets = [x[0] for x in s]
#########################################

In [65]:
# Create a second dataframe to put important information
tweets_final = pd.DataFrame(columns = ["created_at", "id", "in_reply_to_screen_name", "in_reply_to_status_id", "in_reply_to_user_id",
                                      "retweeted_id", "retweeted_screen_name", "user_mentions_screen_name", "user_mentions_id", 
                                       "text", "user_id", "screen_name", "followers_count"])

In [66]:
# Columns that are going to be the same
equal_columns = ["created_at", "id", "text"]
tweets_final[equal_columns] = tweets_df[equal_columns]

In [67]:
# Get the basic information about user 
def get_basics(tweets_final):
    print(tweets_df["user"])
    tweets_final["screen_name"] = tweets_df["user"].apply(lambda x: x["screen_name"])
    tweets_final["user_id"] = tweets_df["user"].apply(lambda x: x["id"])
    tweets_final["followers_count"] = tweets_df["user"].apply(lambda x: x["followers_count"])
    return tweets_final

In [68]:
# Get the user mentions 
def get_usermentions(tweets_final):
    # Inside the tag 'entities' will find 'user mentions' and will get 'screen name' and 'id'
    tweets_final["user_mentions_screen_name"] = tweets_df["entities"].apply(lambda x: x["user_mentions"][0]["screen_name"] if x["user_mentions"] else np.nan)
    tweets_final["user_mentions_id"] = tweets_df["entities"].apply(lambda x: x["user_mentions"][0]["id_str"] if x["user_mentions"] else np.nan)
    return tweets_final

In [69]:
# Get retweets
def get_retweets(tweets_final):
    # Inside the tag 'retweeted_status' will find 'user' and will get 'screen name' and 'id'    
    tweets_final["retweeted_screen_name"] = tweets_df["retweeted_status"].apply(lambda x: x["user"]["screen_name"] if x is not np.nan else np.nan)
    tweets_final["retweeted_id"] = tweets_df["retweeted_status"].apply(lambda x: x["user"]["id_str"] if x is not np.nan else np.nan)
    return tweets_final

In [70]:
# Get the information about replies
def get_in_reply(tweets_final):
    # Just copy the 'in_reply' columns to the new dataframe
    tweets_final["in_reply_to_screen_name"] = tweets_df["in_reply_to_screen_name"]
    tweets_final["in_reply_to_status_id"] = tweets_df["in_reply_to_status_id"]
    tweets_final["in_reply_to_user_id"]= tweets_df["in_reply_to_user_id"]
    return tweets_final

In [71]:
# Lastly fill the new dataframe with the important information
def fill_df(tweets_final):
    get_basics(tweets_final)
    get_usermentions(tweets_final)
    get_retweets(tweets_final)
    get_in_reply(tweets_final)
    return tweets_final

In [72]:
# Get the interactions between the different users
def get_interactions(row):
    # From every row of the original dataframe
    # First we obtain the 'user_id' and 'screen_name'
    user = row["user_id"], row["screen_name"]
    # Be careful if there is no user id
    if user[0] is None:
        return (None, None), []
    
    # The interactions are going to be a set of tuples
    interactions = set()
    
    # Add all interactions 
    # First, we add the interactions corresponding to replies adding the id and screen_name
    interactions.add((row["in_reply_to_user_id"], row["in_reply_to_screen_name"]))
    # After that, we add the interactions with retweets
    interactions.add((row["retweeted_id"], row["retweeted_screen_name"]))
    # And later, the interactions with user mentions
    interactions.add((row["user_mentions_id"], row["user_mentions_screen_name"]))
    
    # Discard if user id is in interactions
    interactions.discard((row["user_id"], row["screen_name"]))
    # Discard all not existing values
    interactions.discard((None, None))
    # Return user and interactions
    return user, interactions

In [73]:
tweets_final = fill_df(tweets_final)

0     {'id': 1263440113033347072, 'id_str': '1263440...
1     {'id': 2846719095, 'id_str': '2846719095', 'na...
2     {'id': 748148699604357121, 'id_str': '74814869...
3     {'id': 460375090, 'id_str': '460375090', 'name...
4     {'id': 810862695582994432, 'id_str': '81086269...
5     {'id': 69687111, 'id_str': '69687111', 'name':...
6     {'id': 96206666, 'id_str': '96206666', 'name':...
7     {'id': 2821451759, 'id_str': '2821451759', 'na...
8     {'id': 59469228, 'id_str': '59469228', 'name':...
9     {'id': 4792465401, 'id_str': '4792465401', 'na...
10    {'id': 2679924194, 'id_str': '2679924194', 'na...
11    {'id': 3331008561, 'id_str': '3331008561', 'na...
12    {'id': 3059457425, 'id_str': '3059457425', 'na...
13    {'id': 1119057737747456005, 'id_str': '1119057...
14    {'id': 40042258, 'id_str': '40042258', 'name':...
15    {'id': 42645874, 'id_str': '42645874', 'name':...
16    {'id': 1195007817176010754, 'id_str': '1195007...
17    {'id': 245385426, 'id_str': '245385426', '

In [74]:
tweets_final = tweets_final.where((pd.notnull(tweets_final)), None)

In [75]:
graph = nx.Graph()

In [76]:
for index, tweet in tweets_final.iterrows():
    user, interactions = get_interactions(tweet)
    user_id, user_name = user
    tweet_id = int(tweet["id"])
    #tweet_sent = tweet["sentiment"]
    for interaction in interactions:
        int_id, int_name = interaction
        graph.add_edge(user_id, int_id, tweet_id=tweet_id)
        

        graph.node[user_id]["name"] = user_name
        graph.node[user_id]["text"] = tweet['text']
        graph.node[int_id]["name"] = int_name
        graph.node[int_id]["text"] = tweet['text']

In [77]:
degrees = [val for (node, val) in graph.degree()]

In [78]:
if nx.is_connected(graph):
    print("The graph is connected")
else:
    print("The graph is not connected")

The graph is not connected


In [79]:
sorted(nx.connected_component_subgraphs(graph), key=len)[-1].number_of_nodes()

4

In [103]:
largest_subgraph = max(nx.connected_component_subgraphs(graph), key=len)

graph_centrality = nx.degree_centrality(largest_subgraph)

max_de = max(graph_centrality.items(), key=itemgetter(1))

graph_closeness = nx.closeness_centrality(largest_subgraph)

max_clo = max(graph_closeness.items(), key=itemgetter(1))

graph_betweenness = nx.betweenness_centrality(largest_subgraph, normalized=True, endpoints=False)

max_bet = max(graph_betweenness.items(), key=itemgetter(1))

all_bet = sorted(graph_betweenness.items(), key=itemgetter(1))

all_bet.reverse()

all_de = sorted(graph_centrality.items(), key=itemgetter(1))
all_de.reverse()

ids = []
for de in all_de:
#     print(graph.node[de[0]])
    ids.append(de[0])
#     print(de[0])
#     print(graph.node[de[0]]['name'])

ids = ids[:10]

users = api.lookup_users(user_ids=ids)


user_list = []
for user in users:
    user_list.append(user._json)

##############################################
influencers = user_list
##############################################


In [104]:
print(hashtag_wordclouds)
print(news_articles)
print(viral_tweets)
print(influencers)

hashtags_len = len(hashtags)
reports = nexus.reports.find({"hashtags": {"$size": hashtags_len, "$all": hashtags}});

for r in reports:
    report_id = r['id']
    r['hashtag_wordclouds'] = hashtag_wordclouds
    r['news_articles'] = news_articles
    r['viral_tweets'] = viral_tweets
    r['influencers'] = influencers
    nexus.reports.update({'id': report_id}, r, upsert=True)

[{'text': 'insurance', 'value': 34}, {'text': 'fintech', 'value': 4}, {'text': 'Insurtech', 'value': 4}, {'text': 'jobs', 'value': 4}, {'text': 'customer', 'value': 2}, {'text': 'medical', 'value': 2}, {'text': 'MachineIntelligence', 'value': 2}, {'text': 'homeowners', 'value': 2}, {'text': 'startup', 'value': 2}, {'text': 'insurtech', 'value': 1}, {'text': 'FinTech', 'value': 1}, {'text': 'BPO', 'value': 1}, {'text': 'fortune500', 'value': 1}, {'text': 'healthcare', 'value': 1}, {'text': 'menstrualdisorders', 'value': 1}, {'text': 'srhr', 'value': 1}, {'text': 'MHH', 'value': 1}, {'text': 'IoT', 'value': 1}, {'text': 'employers', 'value': 1}, {'text': 'sec', 'value': 1}, {'text': 'last', 'value': 1}, {'text': 'minute', 'value': 1}, {'text': 'terms', 'value': 1}, {'text': 'businessowners', 'value': 1}, {'text': 'businessinterruption', 'value': 1}, {'text': 'covid19', 'value': 1}, {'text': 'travel', 'value': 1}, {'text': 'rauschinsuranceservices', 'value': 1}, {'text': 'insurancenews', 

In [105]:
li = [['a', 'b'], ['a'], ['b', 'a'] ]

In [108]:
u = [list(x) for x in set(tuple(sorted(x)) for x in li)]

In [109]:
u

[['a'], ['a', 'b']]

In [7]:
hashtags = ['#insurance']

hashtags_len = len(hashtags)
reports = nexus.reports.find({"hashtags": {"$size": hashtags_len, "$all": hashtags}});

for r in reports:
    report_id = r['id']
    r['']
    nexus.reports.update({'id': report_id}, r, upsert=True)
    
    
    





In [ ]:
nexus.reports.update({} )

In [36]:
def update_report():
#     logger.info(str('updated report task called.'))

    myclient = MongoClient()
    nexus = myclient["nexus"]
    tweets = nexus["tweets"]
    users = nexus["users"]
    reports = nexus["reports"]

    reps = reports.find({'user.sub': 'google-oauth2|110224375451341506987'})

    hashtags_list = []
    for r in reps:
        hashtags_list.append(r['hashtags'])

    unique_data = [list(x) for x in set(tuple(sorted(x)) for x in hashtags_list)]
    
    print('unique hashtag list:', unique_data)

    for hashtags in unique_data:
        queries = []
        for hashtag in hashtags:
            hashtag = hashtag[1:]
            queries.append({"entities.hashtags.text": hashtag})
        query = [{'$match': {'$or': queries}}]
        tweets = nexus.tweets.aggregate(query)

        tweet_list = []
        for tweet in tweets:
            tweet_list.append(tweet)

        tweets_df = pd.DataFrame(tweet_list)

        all_hashtags = []
        for e in tweets_df.entities:
            hashtags = [t['text'] for t in e['hashtags']]
            all_hashtags = all_hashtags + hashtags

        from collections import Counter
        c = Counter(all_hashtags)
        most_common_tuples = c.most_common()
        sorted_keys = sorted(c, key=c.get, reverse=True)

        hashtag_dict = {}
        for t in most_common_tuples:
            hashtag_dict[t[0]] = t[1]

        hashtag_wordclouds = []
        for t in most_common_tuples:
            hashtag_wordclouds.append({'text': t[0], 'value': t[1]})

        for t in most_common_tuples:
            tweets_df['#' + t[0]] = False

        for i, e in enumerate(tweets_df.entities):
            hashtags = [t['text'] for t in e['hashtags']]
            for hashtag in hashtags:
                tweets_df.at[i, '#' + hashtag] = True

        def apply_func(x):
            if not isinstance(x, float):
                if 'full_text' in x:
                    return x['full_text']
                else:
                    return float('nan')
            else:
                return float('nan')

        tweets_df['full_text'] = tweets_df['extended_tweet'].apply(lambda x: apply_func(x))
        tweets_df.full_text.fillna(tweets_df.text, inplace=True)

        import re
        from collections import Counter
        import requests

        import extraction
        import requests

        import favicon

        url_list = []
        for text in tweets_df['text'].values.tolist():
            urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
            url_list += urls

        url_list = list(filter(lambda url: len(url) > 13, url_list))

        c = Counter(url_list)
        most_common_urls = c.most_common()
        sorted_urls = sorted(c, key=c.get, reverse=True)

        news_articles = []
        twitter_domain_url = 'https://twitter.com'

        url_list = []
        for e in tweets_df.entities:
            for url_obj in e['urls']:
                if url_obj['expanded_url'][:19] != 'https://twitter.com':
                    url_list.append(url_obj['expanded_url'])

        c = Counter(url_list)
        most_common_urls = c.most_common()
        sorted_urls = sorted(c, key=c.get, reverse=True)

        for i, url in enumerate(sorted_urls):
            print('count value:', i)
            if i == 20:
                break
            try:
                news_article_dict = {}

                html = requests.get(url).text
                extracted = extraction.Extractor().extract(html, source_url=url)
                icon_url = favicon.get(url)[0][0]

                if url[:19] != twitter_domain_url:
                    print('title:', extracted.title)
                    print('description:', extracted.description)
                    print(extracted.image, url, icon_url)

                    news_article_dict['title'] = extracted.title
                    news_article_dict['description'] = extracted.description
                    news_article_dict['favicon'] = icon_url
                    news_article_dict['image'] = extracted.image
                    news_article_dict['url'] = url
                    news_article_dict['share_count'] = most_common_urls[i][1]

                    news_articles.append(news_article_dict)

                print('\n')
            except:
                print(url)
                print('\n')

        # Accesing twitter from the App created in my account
        def autorize_twitter_api():
            """
            This function gets the consumer key, consumer secret key, access token
            and access token secret given by the app created in your Twitter account
            and authenticate them with Tweepy.
            """
            # Get access and costumer key and tokens
            auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_token_secret)

            return auth

        api = tweepy.API(wait_on_rate_limit_notify=True, wait_on_rate_limit=True, auth_handler=autorize_twitter_api())

        retweeted_ids = []
        for rt in tweets_df['retweeted_status'].values:
            if not isinstance(rt, float):
                retweeted_ids.append(rt['id'])

        c = Counter(retweeted_ids)
        most_common_tuples = c.most_common()
        sorted_keys = sorted(c, key=c.get, reverse=True)
        most_common_tuples

        ids = []
        for t in most_common_tuples:
            ids.append(t[0])
        ids = ids[:20]

        statuses = api.statuses_lookup(ids)

        statuses_list = []
        for status in statuses:
            t_obj = {
                'text': status._json['text'],
                'id': status._json['id'],
                'tweet_link': 'https://twitter.com/i/web/status/' + status._json['id_str'],
                'user_screen_name': status._json['user']['screen_name'],
                'json': status._json
            }

            t = (t_obj, status._json['retweet_count'])
            statuses_list.append(t)

        s = sorted(statuses_list, key=lambda x: x[1])
#         s.reverse()

        viral_tweets = [x[0] for x in s]

        # Create a second dataframe to put important information
        tweets_final = pd.DataFrame(
            columns=["created_at", "id", "in_reply_to_screen_name", "in_reply_to_status_id", "in_reply_to_user_id",
                     "retweeted_id", "retweeted_screen_name", "user_mentions_screen_name", "user_mentions_id",
                     "text", "user_id", "screen_name", "followers_count"])

        # Columns that are going to be the same
        equal_columns = ["created_at", "id", "text"]
        tweets_final[equal_columns] = tweets_df[equal_columns]

        # Get the basic information about user
        def get_basics(tweets_final):
            print(tweets_df["user"])
            tweets_final["screen_name"] = tweets_df["user"].apply(lambda x: x["screen_name"])
            tweets_final["user_id"] = tweets_df["user"].apply(lambda x: x["id"])
            tweets_final["followers_count"] = tweets_df["user"].apply(lambda x: x["followers_count"])
            return tweets_final

        # Get the user mentions
        def get_usermentions(tweets_final):
            # Inside the tag 'entities' will find 'user mentions' and will get 'screen name' and 'id'
            tweets_final["user_mentions_screen_name"] = tweets_df["entities"].apply(
                lambda x: x["user_mentions"][0]["screen_name"] if x["user_mentions"] else np.nan)
            tweets_final["user_mentions_id"] = tweets_df["entities"].apply(
                lambda x: x["user_mentions"][0]["id_str"] if x["user_mentions"] else np.nan)
            return tweets_final

        # Get retweets
        def get_retweets(tweets_final):
            # Inside the tag 'retweeted_status' will find 'user' and will get 'screen name' and 'id'
            tweets_final["retweeted_screen_name"] = tweets_df["retweeted_status"].apply(
                lambda x: x["user"]["screen_name"] if x is not np.nan else np.nan)
            tweets_final["retweeted_id"] = tweets_df["retweeted_status"].apply(
                lambda x: x["user"]["id_str"] if x is not np.nan else np.nan)
            return tweets_final

        # Get the information about replies
        def get_in_reply(tweets_final):
            # Just copy the 'in_reply' columns to the new dataframe
            tweets_final["in_reply_to_screen_name"] = tweets_df["in_reply_to_screen_name"]
            tweets_final["in_reply_to_status_id"] = tweets_df["in_reply_to_status_id"]
            tweets_final["in_reply_to_user_id"] = tweets_df["in_reply_to_user_id"]
            return tweets_final

        # Lastly fill the new dataframe with the important information
        def fill_df(tweets_final):
            get_basics(tweets_final)
            get_usermentions(tweets_final)
            get_retweets(tweets_final)
            get_in_reply(tweets_final)
            return tweets_final

        # Get the interactions between the different users
        def get_interactions(row):
            # From every row of the original dataframe
            # First we obtain the 'user_id' and 'screen_name'
            user = row["user_id"], row["screen_name"]
            # Be careful if there is no user id
            if user[0] is None:
                return (None, None), []

            # The interactions are going to be a set of tuples
            interactions = set()

            # Add all interactions
            # First, we add the interactions corresponding to replies adding the id and screen_name
            interactions.add((row["in_reply_to_user_id"], row["in_reply_to_screen_name"]))
            # After that, we add the interactions with retweets
            interactions.add((row["retweeted_id"], row["retweeted_screen_name"]))
            # And later, the interactions with user mentions
            interactions.add((row["user_mentions_id"], row["user_mentions_screen_name"]))

            # Discard if user id is in interactions
            interactions.discard((row["user_id"], row["screen_name"]))
            # Discard all not existing values
            interactions.discard((None, None))
            # Return user and interactions
            return user, interactions

        tweets_final = fill_df(tweets_final)

        tweets_final = tweets_final.where((pd.notnull(tweets_final)), None)

        graph = nx.Graph()

        for index, tweet in tweets_final.iterrows():
            user, interactions = get_interactions(tweet)
            user_id, user_name = user
            tweet_id = int(tweet["id"])
            # tweet_sent = tweet["sentiment"]
            for interaction in interactions:
                int_id, int_name = interaction
                graph.add_edge(user_id, int_id, tweet_id=tweet_id)

                graph.node[user_id]["name"] = user_name
                graph.node[user_id]["text"] = tweet['text']
                graph.node[int_id]["name"] = int_name
                graph.node[int_id]["text"] = tweet['text']

        degrees = [val for (node, val) in graph.degree()]

        largest_subgraph = max(nx.connected_component_subgraphs(graph), key=len)

        graph_centrality = nx.degree_centrality(largest_subgraph)

        max_de = max(graph_centrality.items(), key=itemgetter(1))

        graph_closeness = nx.closeness_centrality(largest_subgraph)

        max_clo = max(graph_closeness.items(), key=itemgetter(1))

        graph_betweenness = nx.betweenness_centrality(largest_subgraph, normalized=True, endpoints=False)

        max_bet = max(graph_betweenness.items(), key=itemgetter(1))

        all_bet = sorted(graph_betweenness.items(), key=itemgetter(1))

        all_bet.reverse()

        all_de = sorted(graph_centrality.items(), key=itemgetter(1))
        all_de.reverse()

        ids = []
        for de in all_de:
            #     print(graph.node[de[0]])
            ids.append(de[0])
        #     print(de[0])
        #     print(graph.node[de[0]]['name'])

        ids = ids[:10]

        users = api.lookup_users(user_ids=ids)

        user_list = []
        for user in users:
            user_list.append(user._json)

        ##############################################
        influencers = user_list
        ##############################################



#         print(hashtag_wordclouds)
#         print(news_articles)
#         print(viral_tweets)
#         print(influencers)

        hashtags_len = len(hashtags)
        print('hashtags:', hashtags)
        hashtags = ['#'+hashtag for hashtag in hashtags]
        
        rprts = nexus.reports.find({"hashtags": {"$size": hashtags_len, "$all": hashtags}})
        

        for r in rprts:
            report_id = r['id']
            print('report_id:', r['id'])
            print('report:', r)
            
            r['hashtag_wordclouds'] = hashtag_wordclouds
            r['news_articles'] = news_articles
            r['viral_tweets'] = viral_tweets
            r['influencers'] = influencers

            print('updating reports...')
            nexus.reports.update({'id': report_id}, r, upsert=True)

In [37]:
update_report()

unique hashtag list: [['#insurance'], ['#plandemic']]
count value: 0
title: Insurtech
description: None
None https://insurtechnttdata.everis.com/?lang=en&utm_campaign=Insurtech&utm_medium=email&utm_source=Revue%20newsletter https://insurtechnttdata.everis.com/favicon.ico


count value: 1
title: Not Acceptable!
description: An appropriate representation of the requested resource could not be found on this server. This error was generated by Mod_Security.
None https://www.premiershieldinsurance.com/insurance-quotes/auto https://www.premiershieldinsurance.com/wp-content/uploads/2019/03/cropped-PSI-Shield-Only-Logo-270x270.jpg


count value: 2
title: Riskcovry closes pre-series A round on INR 9.3 crore ($1.2m) | FinTech Global
description: Digital Insurance platform Riskcovry has reportedly closed its pre-Series A round on INR 9.3 crore ($1.2m).
https://member.fintech.global/wp-content/uploads/sites/3/2020/05/photo-1525466760727-1d8be8721154.jpg https://bit.ly/3d4YZBd https://analysis.cair

hashtags: ['insurance']
report_id: c99d6cf3-6b67-4884-9c20-68941a4abde4
report: {'_id': ObjectId('5ece84b1ac456b00e44c2180'), 'id': 'c99d6cf3-6b67-4884-9c20-68941a4abde4', 'user': {'given_name': 'John', 'family_name': 'Joy', 'nickname': 'johnjoykurian', 'name': 'John Joy', 'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g', 'locale': 'en', 'updated_at': '2020-05-27T15:15:19.581Z', 'email': 'johnjoykurian@gmail.com', 'email_verified': True, 'sub': 'google-oauth2|110224375451341506987'}, 'hashtags': ['#insurance']}
updating reports...
report_id: 618c9db9-37be-4351-a8af-0e9a30dfad3a
report: {'_id': ObjectId('5ece850bac456b00e44c21b5'), 'id': '618c9db9-37be-4351-a8af-0e9a30dfad3a', 'user': {'given_name': 'John', 'family_name': 'Joy', 'nickname': 'johnjoykurian', 'name': 'John Joy', 'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g', 'locale': 'en', 'updated_at': '2020-05-27T15:15:19.581Z', 'email': 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:351: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


title: Nurse 👨🏾‍⚕️ Jigga #NWO Virus 🦠
description: 
https://i.ytimg.com/vi/0Ahsd1myFEo/maxresdefault.jpg https://youtu.be/0Ahsd1myFEo https://www.youtube.com/yts/img/favicon_144-vfliLAfaB.png


0     {'id': 953411717890887680, 'id_str': '95341171...
1     {'id': 34657722, 'id_str': '34657722', 'name':...
2     {'id': 31288651, 'id_str': '31288651', 'name':...
3     {'id': 1073450645137440768, 'id_str': '1073450...
4     {'id': 1259581401525096450, 'id_str': '1259581...
5     {'id': 1261881678588411905, 'id_str': '1261881...
6     {'id': 608033428, 'id_str': '608033428', 'name...
7     {'id': 1018897563230064641, 'id_str': '1018897...
8     {'id': 173897184, 'id_str': '173897184', 'name...
9     {'id': 1010694724934602752, 'id_str': '1010694...
10    {'id': 604016671, 'id_str': '604016671', 'name...
11    {'id': 1240049950748278786, 'id_str': '1240049...
12    {'id': 823010621935456256, 'id_str': '82301062...
Name: user, dtype: object
hashtags: ['plandemic']
report_id: c44f5014-c899-4c2

In [38]:
reports = nexus.reports.find({});

In [14]:
hashtag_wordclouds

NameError: name 'hashtag_wordclouds' is not defined

In [39]:
for r in reports:
    print(r)

{'_id': ObjectId('5ece84b1ac456b00e44c2180'), 'id': 'c99d6cf3-6b67-4884-9c20-68941a4abde4', 'user': {'given_name': 'John', 'family_name': 'Joy', 'nickname': 'johnjoykurian', 'name': 'John Joy', 'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g', 'locale': 'en', 'updated_at': '2020-05-27T15:15:19.581Z', 'email': 'johnjoykurian@gmail.com', 'email_verified': True, 'sub': 'google-oauth2|110224375451341506987'}, 'hashtags': ['#insurance'], 'hashtag_wordclouds': [{'text': 'insurance', 'value': 46}, {'text': 'fintech', 'value': 4}, {'text': 'Insurtech', 'value': 4}, {'text': 'jobs', 'value': 4}, {'text': 'customer', 'value': 2}, {'text': 'medical', 'value': 2}, {'text': 'covid19', 'value': 2}, {'text': 'MachineIntelligence', 'value': 2}, {'text': 'homeowners', 'value': 2}, {'text': 'startup', 'value': 2}, {'text': 'insurancenews', 'value': 2}, {'text': 'MA', 'value': 2}, {'text': 'cheapcarinsurance', 'value': 2}, {'text': 'insurtech', 'value': 1},

In [15]:
r

{'_id': ObjectId('5ed0ebaa9812d348a34a67cc'),
 'id': 'c44f5014-c899-4c28-8651-ba8f68715838',
 'user': {'given_name': 'John',
  'family_name': 'Joy',
  'nickname': 'johnjoykurian',
  'name': 'John Joy',
  'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g',
  'locale': 'en',
  'updated_at': '2020-05-29T10:48:22.506Z',
  'email': 'johnjoykurian@gmail.com',
  'email_verified': True,
  'sub': 'google-oauth2|110224375451341506987'},
 'hashtags': ['#plandemic']}

In [16]:
r['wordcloud'] = {'some': 'thing'}
nexus.reports.update({'id': 'c44f5014-c899-4c28-8651-ba8f68715838'}, r, upsert=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [18]:
for r in nexus.reports.find({}):
    print(r)

{'_id': ObjectId('5ece84b1ac456b00e44c2180'), 'id': 'c99d6cf3-6b67-4884-9c20-68941a4abde4', 'user': {'given_name': 'John', 'family_name': 'Joy', 'nickname': 'johnjoykurian', 'name': 'John Joy', 'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g', 'locale': 'en', 'updated_at': '2020-05-27T15:15:19.581Z', 'email': 'johnjoykurian@gmail.com', 'email_verified': True, 'sub': 'google-oauth2|110224375451341506987'}, 'hashtags': ['#insurance']}
{'_id': ObjectId('5ece850bac456b00e44c21b5'), 'id': '618c9db9-37be-4351-a8af-0e9a30dfad3a', 'user': {'given_name': 'John', 'family_name': 'Joy', 'nickname': 'johnjoykurian', 'name': 'John Joy', 'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g', 'locale': 'en', 'updated_at': '2020-05-27T15:15:19.581Z', 'email': 'johnjoykurian@gmail.com', 'email_verified': True, 'sub': 'google-oauth2|110224375451341506987'}, 'hashtags': ['#insurance']}
{'_id': ObjectId('5ece8606ac45

In [20]:
hashtags = ['#plandemic']
reports = nexus.reports.find({"hashtags": {"$size": hashtags_len, "$all": hashtags}})

for r in reports:
    print(r)

{'_id': ObjectId('5ed0ebaa9812d348a34a67cc'), 'id': 'c44f5014-c899-4c28-8651-ba8f68715838', 'user': {'given_name': 'John', 'family_name': 'Joy', 'nickname': 'johnjoykurian', 'name': 'John Joy', 'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g', 'locale': 'en', 'updated_at': '2020-05-29T10:48:22.506Z', 'email': 'johnjoykurian@gmail.com', 'email_verified': True, 'sub': 'google-oauth2|110224375451341506987'}, 'hashtags': ['#plandemic'], 'wordcloud': {'some': 'thing'}}


In [43]:
report = nexus.reports.find_one({'id': '5eeda95b-7a94-4371-abbe-eca21b7cf11b'})
del report['_id']
print(report)

{'id': '5eeda95b-7a94-4371-abbe-eca21b7cf11b', 'user': {'given_name': 'John', 'family_name': 'Joy', 'nickname': 'johnjoykurian', 'name': 'John Joy', 'picture': 'https://lh3.googleusercontent.com/a-/AOh14Gghoo5HSHaUEAa6EHB7XGaFUU0uFC1HrZPUjA1V1g', 'locale': 'en', 'updated_at': '2020-05-27T15:15:19.581Z', 'email': 'johnjoykurian@gmail.com', 'email_verified': True, 'sub': 'google-oauth2|110224375451341506987'}, 'hashtags': ['#insurance'], 'hashtag_wordclouds': [{'text': 'insurance', 'value': 46}, {'text': 'fintech', 'value': 4}, {'text': 'Insurtech', 'value': 4}, {'text': 'jobs', 'value': 4}, {'text': 'customer', 'value': 2}, {'text': 'medical', 'value': 2}, {'text': 'covid19', 'value': 2}, {'text': 'MachineIntelligence', 'value': 2}, {'text': 'homeowners', 'value': 2}, {'text': 'startup', 'value': 2}, {'text': 'insurancenews', 'value': 2}, {'text': 'MA', 'value': 2}, {'text': 'cheapcarinsurance', 'value': 2}, {'text': 'insurtech', 'value': 1}, {'text': 'FinTech', 'value': 1}, {'text': 'B